In [50]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from collections import Counter


In [30]:
property_df = pd.read_csv("../data/curated/property_isochromes.csv")

In [31]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,https://www.domain.com.au/406a-62-66-nicholson...,406A/62 - 66 Nicholson Street Footscray VIC 3011,280.0,1,1,0,Apartment / Unit / Flat,locat matter,213031348,144.899514,-37.798485,213031348.0,916.0,355.0,2296.0,1763.0,16695.0,Footscray,10.0


In [51]:
y = property_df['cost_text'].tolist()
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X = property_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021"], axis=1)
X.astype({'cost_text': 'float', 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "int64", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(280.0, <class 'numpy.float64'>), (1, <class 'numpy.int64'>), (1, <class 'numpy.int64'>), (0, <class 'numpy.int64'>), ('Apartment / Unit / Flat', <class 'str'>), (916.0, <class 'numpy.float64'>), (355.0, <class 'numpy.float64'>), (2296.0, <class 'numpy.float64'>), (1763.0, <class 'numpy.float64'>), (16695.0, <class 'numpy.float64'>), ('Footscray', <class 'str'>), (10.0, <class 'numpy.float64'>)]


,cost_text,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,280.0,1,1,0,Apartment / Unit / Flat,916.0,355.0,2296.0,1763.0,16695.0,Footscray,10.0


In [52]:
type(y[0])

float

In [64]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = ['cost_text', 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "duration_mins", 
                  "Bed", 
                  "Bath", 
                  "Park"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPRegressor(hidden_layer_sizes=[100, 10, 4], max_iter=2000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)])
print('Cross-val noncheating standardised features acc:', np.mean(cross_val_score(pipeline, X, y, cv=10)))



Cross-val noncheating standardised features acc: 0.9999660496990043


In [63]:
clf.fit(preprocessor.fit_transform(X),y) # need to perform a train test split here
# need to do balanced split- i.e. one of the bootstrapping methods or something to make sure all suburbs have
# a good amount of representation. 


#print(clf.coefs_)
print('parameter shapes:',[p.shape for p in clf.coefs_])
print('num layers:', clf.n_layers_)

# next steps:
# build growth models for crime, income, pop, etc by suburb 
# predict these rates in a certain year
# input it into a model along with a suburb for a "Standard" house with however many beds/parking/baths etc.
# predict. 

[array([[ 1.21751748e+00,  1.56936923e+00,  1.86998413e+00,
         1.29104084e+00,  1.55688491e+00,  1.29200214e+00,
         1.72803854e+00,  1.78592941e+00,  1.47492065e+00,
         1.21380850e+00],
       [-5.99268186e-01,  4.10418455e-01,  1.03241386e-01,
        -7.09256427e-02, -3.82253148e-03,  1.56131961e-01,
        -1.09180707e-01,  2.02812887e-01,  2.56081471e-01,
         2.23233482e-01],
       [ 4.14981171e-01,  7.72337952e-01,  1.37844974e-02,
         1.00905939e-01,  1.50822219e-01,  5.77284120e-02,
        -2.50751560e-01,  5.23168174e-02, -5.63049527e-01,
         3.52860523e-02],
       [-9.82154483e-02,  5.80823580e-01, -7.44134726e-02,
         1.81931397e-01, -2.05255947e-01, -1.03169574e-01,
         2.34876922e-02, -1.38734571e-01,  4.75833028e-01,
         6.83336603e-02],
       [ 1.39693807e-01, -1.67631173e-01, -3.56745721e-02,
        -1.66156698e-01, -4.48114726e-02, -2.19050459e-02,
         2.31834645e-01, -7.76076292e-02, -5.74094972e-02,
        -9